# XGBoost

In [1]:
from sklearn.model_selection import KFold
from IPython.display import display
from ipywidgets import IntProgress
import xgboost as xgb
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

In [3]:
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))

In [4]:
calcular_auc = '"{}"'.format(os.path.join(current_folder, '..', 'Calcular AUC.ipynb'))

In [5]:
set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))

In [6]:
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

In [7]:
hiperparametros_csv = os.path.join(current_folder, 'hiperparametros', 'xgboost.csv')

Cargo el df con los features.

In [8]:
pd.options.mode.chained_assignment = None

In [9]:
%run $merge_features

[3.93356, 3.49488]


KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [10]:
assert(df_features.shape[0] == df['person'].unique().shape[0])

Cargo los sets de entrenamiento, testing y predicción.

In [11]:
%run $set_de_entrenamiento_testing_y_prediccion

## Entrenamiento rápido

In [12]:
training = labels_training.merge(df_features, how='inner', on='person')

In [13]:
training.head()

,label,screen_resolution_height,screen_resolution_width,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,...,eventos noche_y,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,noche,maniana,madrugada,tarde
person,,,,,,,,,,,,,,,,,,,,,,,,,
6ec7ee77,0,640.000000,360.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
abe7a2fb,0,640.000000,360.000000,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,0.0,...,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
34728364,0,640.000000,360.000000,0.0,5.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
db2c4d27,1,640.491228,429.140351,3.0,31.0,3.0,1.0,166.0,0.0,92.0,13.0,20.0,...,250,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1
cde431db,0,427.000000,320.000000,2.0,1.0,4.0,1.0,0.0,0.0,1.0,2.0,2.0,...,8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0,1


In [14]:
columns = list(training.columns)
columns.remove('label')

In [15]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

Hiperparámetros:

In [16]:
param = {'max_depth': 4, 'eta': 0.54, 'gamma': 7, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

In [17]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

Predigo:

In [18]:
labels_test = labels_test.merge(df_features, how='inner', on='person')

In [19]:
test_matrix = xgb.DMatrix(labels_test.loc[:, columns])

In [20]:
labels_test['label_predicted'] = bst.predict(test_matrix)

Métrica de la predicción:

In [21]:
%run $calcular_auc

auc score: 0.8487966044655322


## Búsqueda de hiperparámetros con k-fold

Vamos a hacer un kfold con sklearn.

In [22]:
splits = 5
max_depth_values = 5
eta_values = 5
gamma_values = 5
num_round_values = 5

In [23]:
param = {
    'max_depth': 0,
    'eta': 0,
    'gamma': 0,
    'silent': 1,
    'objective': 'binary:logistic',
    'nthread': 10,
    'eval_metric': 'auc'
}

In [24]:
labels_with_features = labels.merge(df_features, how='inner', on='person')

In [25]:
columns = list(labels_with_features.columns)
columns.remove('label')

In [26]:
f = IntProgress(min=0, max=splits*max_depth_values*eta_values*gamma_values*num_round_values)
display(f) # display the bar

kf = KFold(n_splits=splits, shuffle=False)
results = pd.DataFrame(columns=['k', 'max_depth', 'eta', 'gamma', 'num_round', 'auc'])
index = 0
k = 0
for train_index, test_index in kf.split(labels):
    
    labels_training = labels_with_features.iloc[train_index]
    labels_test = labels_with_features.iloc[test_index]
    assert(labels_training.merge(labels_test, how='inner', on='person').shape[0] == 0)
    train_matrix = xgb.DMatrix(labels_training.loc[:, columns], label=labels_training['label'])
    test_matrix = xgb.DMatrix(labels_test.loc[:, columns])
    
    for max_depth, eta, gamma, num_round in np.ndindex((max_depth_values, eta_values, gamma_values, num_round_values)):
        eta=eta/eta_values
        param['max_depth'] = max_depth
        param['eta'] = eta
        param['gamma'] = gamma
        
        bst = xgb.train(param, train_matrix, num_round)
        labels_test['label_predicted'] = bst.predict(test_matrix)
        
        results.loc[index] = k, max_depth, eta, gamma, num_round, calculate_auc()
        
        index+=1
        f.value += 1
    
    k += 1

IntProgress(value=0, max=3125)

Promedio los hiperparámetros.

In [27]:
results_mean = results.groupby(['max_depth', 'eta', 'gamma', 'num_round'])[['auc']].mean()

In [28]:
mejor_resultado = results_mean.loc[results_mean.idxmax()]
mejor_resultado

,,,,auc
max_depth,eta,gamma,num_round,
4.0,0.8,1.0,4.0,0.841006


Escribo los nuevos resultados en un archivo.

In [29]:
mejores_resultados = pd.read_csv(hiperparametros_csv, index_col='fecha')

In [30]:
mejores_resultados

,max_depth,eta,gamma,k,num_round,auc,features
fecha,,,,,,,
2018-11-11 10:25,4.0,0.6,6.0,10,10,0.859342,"screen_resolution_height,screen_resolution_wid..."


In [31]:
mejor_resultado['k'] = splits
mejor_resultado['features'] = ','.join(columns)
mejor_resultado['fecha'] = pd.datetime.now().strftime("%Y-%m-%d %H:%M")
mejor_resultado.reset_index(inplace=True)
mejor_resultado = mejor_resultado[['fecha', 'auc', 'max_depth', 'eta', 'gamma', 'k', 'num_round', 'features']]
mejor_resultado.set_index('fecha', inplace=True)

In [32]:
mejores_resultados = mejores_resultados.append(mejor_resultado, sort=False)

In [34]:
mejores_resultados.to_csv(hiperparametros_csv)

## Predecir labels desconocidos

In [35]:
training = labels.merge(df_features, how='inner', on='person')

In [36]:
columns = list(training.columns)
columns.remove('label')

In [37]:
dtrain = xgb.DMatrix(training.loc[:, columns], label=training['label'])

In [38]:
param['max_depth'] = 4
param['eta'] = 0.6
param['gamma'] = 6

In [39]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

Predigo:

In [40]:
labels_to_predict_with_features = labels_to_predict.merge(df_features, how='inner', on='person')

In [41]:
assert(labels_to_predict.shape[0] == labels_to_predict_with_features.shape[0])

In [42]:
matrix = xgb.DMatrix(labels_to_predict_with_features.loc[:, columns])

In [43]:
labels_to_predict['label'] = bst.predict(matrix)

In [ ]:
# labels_to_predict.to_csv(predicciones_csv)